# **框架详述**

### **环境信息**

环境交互的两个API：

1. reset(usr_conf)

reset会将环境重置为环境配置文件中指定的状态，并且返回初始观测
```python
obs, state = env.reset(usr_conf = usr_conf)
```

2. env.step(act, stop_game = false)

环境会执行传入的act动作指令，完成一次状态转移，并返回新的观测和奖励等信息
```python
frame_no, _obs, score, terminated, truncated, _state = env.step(act, stop_game = false)
```

### **特征处理**

文件`conf.py`中定义了基本的特征结构的大小等基本信息

```python
class Config:

    FEATURES = [
        2,  # 英雄当前坐标归一化 (x,z)
        6,  # 终点相关特征 (可见标志＋方向向量＋绝对坐标＋距离)
        6,  # 与 10 步前位置的相对特征 (同上结构)
        8,  # 八个方向的合法性掩码
    ]

    FEATURE_SPLIT_SHAPE = FEATURES

    # 整体obs维度
    DIM_OF_OBSERVATION = sum(FEATURES) # 22

    # 移动动作方向的维度
    DIM_OF_ACTION_DIRECTION = 8

    # 闪现动作方向的维度
    DIM_OF_TALENT = 8

    # SampleData用于训练智能体的
    # 大小为 2*(22+8)+act(1)+rew(1)+ret(1)+done(1) == 64
    # 注意在DQN中不仅仅需要当前状态s还需要下一状态s'故这里需要2 * (22+8)
    SAMPLE_DIM = 2 * (DIM_OF_OBSERVATION + DIM_OF_ACTION_DIRECTION) + 4


    # 强化学习参数
    GAMMA = 0.9

    EPSILON_MIN = 0.1
    EPSILON_MAX = 1.0
    EPSILON_DECAY = 1e-6

    START_LR = 1e-4

```

在`definition.py`中也定义了相应的数据结构，同样的也是64维的：
```python
SampleData = create_cls(
    "SampleData",
    obs=None,
    _obs=None,
    obs_legal=None,
    _obs_legal=None,
    act=None,
    rew=None,
    ret=None,
    done=None,
)
```

而`ObsData`是Agent的一个状态输入，包含`feature`与`legal_act`；`ActData`则是Agent根据输入经过算法后选择的输出，随后二者组合，结合相应的环境信息，打包成一个`SampleData`返回给Agent训练更新权重。

文件`preprocessor.py`定义了特征处理的方法
```python
def __init__(self) -> None:
    self.move_action_num = 8
    self.reset()

def reset(self):
    self.step_no = 0
    self.cur_pos = (0, 0)
    self.cur_pos_norm = np.array((0, 0))
    self.end_pos = None
    self.is_end_pos_found = False
    self.history_pos = []
    self.bad_move_ids = set()
```



1. `norm`函数归一化处理，将数值v映射到[0,1]

2. `_get_pos_feature`函数，输入`found`，`cur_pos`，`target_pos`，分别为是否发现目标，当前位置，目标的位置(found=1)，或者估计目标位置(found=0)，此时的输入都是未归一化的。该函数返回found，归一化的朝向x，z坐标，目标坐标x，z，以及距离。是一个6维的特征向量。

3. `pb2struct`函数，更新Preprocessor对象内部的状态，`frame_state`内的`obs`结构大致如下：
```python
obs = {
  "frame_state": {
    "step_no": int,  
    "heroes": [{"pos": {"x":float, "z":float}, ...}, ...],
    "organs": [
      {"sub_type": int, "relative_pos": {"l2_distance":str, "direction":str},
       "status":int, "pos":{"x":float,"z":float}},
      ...
    ],
    ...
  },
  ...
}
```
首先取出环境帧号与第一个hero(在该任务下只有一个)的位置，同时维护一个历史位置队列

之后取出organ，包含当前帧的机关信息，`sub_type==4`意味着终点，读取其的相对位置类别，此处结合了`definition.py`中数据结构的定义

若`status != -1`意味着终点可见，直接读取坐标，否则进行预测(满足以下条件预测)
```python
elif (not self.is_end_pos_found) and (
    self.end_pos is None   # 从未预测过
    or self.step_no % 100 == 0   # 每100帧做一次校正
    or self.end_pos_dir != end_pos_dir   # 方向类别发生变化
    or self.end_pos_dis != end_pos_dis   # 距离类别发生变化
):
```
预测如下：
```python
distance = end_pos_dis * 20
theta = DirectionAngles[end_pos_dir]
delta_x = distance * math.cos(math.radians(theta))
delta_z = distance * math.sin(math.radians(theta))

```
随后进行更新预测坐标，以及上一次预测用到的`end_pos_dir`，`end_pos_dis`

4. `get_legal_action()`函数，首先当上次动作之后位置未变，意味着可能撞墙，检测坐标，若上次动作有效，则说明该动作没有移动，加入`bad_move_ids`。随后屏蔽这些`bad_move_ids`，若全部都被屏蔽，则重置，最终返回合法动作掩码

5. `process`函数，最终进行一个特征的拼接得到`feature`，返回`feature`，`legal_action`，`rewards`。之前的`pb2struct`中的`frame_state`是智能体与环境交互直接返回的`state`，通过`preprocessor`类进行一个抽象处理，得到一个可以进入网络预测的序列`ObsData`




### **Agent流程**

代码位于`agent.py`下，是与环境交互的核心

1. 重要成员

    - `self.algorithm`：DQN 算法核心实例，负责 predict_detail、learn 等操作。

    - `self.preprocessor`：前面细讲的 Preprocessor，将原始 obs 转成 (feature, legal_mask, reward) 三元组。

    - `self.last_action`：跟踪上一步动作，用于过滤无效移动。

    - `self.win_history`：记录最近 100 场的胜负，用于统计胜率。

2. 首先使用`reset()`重置环境，

3. 使用`predict`进行预测，提交一批次的`ObsData`，返回`ActData`

4. `exploit`用于推断/线上模式，每次只处理单条的`observation`

5. `learn`接受外部的`SampleData`列表，调用DQN学习方法进行参数更新

6. `save_model`与`load_model`保存/加载模型

7. 两个数据转换函数`observation_process`，`action_process`

```python
def observation_process(self, obs, extra_info):
    (feature_vec, legal_action, reward_list) = self.preprocessor.process([obs, extra_info], self.last_action)
    return ObsData(feature=feature_vec, legal_act=legal_action), reward_list
```
调用`preprocessor`类中的方法得到`ObsData(feature, legal_act)`与`reward_list`
```python
def action_process(self, act_data):
    result = act_data.move_dir
    result += act_data.use_talent * 8
    self.last_action = result
    return result

```
输入一个`ActData`实列，随后合并编码进行映射到[0, 15]，更新`last_action`并返回



### **算法流程**

代码位于`algorithm.py`

1. 统一从`Config`中读取，参数如下，同时构建神经网络+优化器+监控(logger输出)

```python
def __init__(self, device, logger, monitor):
    self.act_shape       = Config.DIM_OF_ACTION_DIRECTION   # 8 个移动方向
    self.direction_space = Config.DIM_OF_ACTION_DIRECTION
    self.talent_direction= Config.DIM_OF_TALENT              # 8 个闪现方向
    self.obs_shape       = Config.DIM_OF_OBSERVATION        # 22 维特征
    self.epsilon_max     = Config.EPSILON_MAX               # ε 最大值 = 1.0
    self.epsilon_min     = Config.EPSILON_MIN               # ε 最小值 = 0.1
    self.epsilon_decay   = Config.EPSILON_DECAY             # ε 衰减速率
    self._gamma          = Config.GAMMA                     # 折扣因子 = 0.9
    self.lr              = Config.START_LR                  # 初始学习率 = 1e-4
    self.device          = device                           # 训练/推理用的设备
```

2. **核心流程`learn`**

传入的`list_sample_data`本质就是`Sample_Data` 的实例
```python
SampleData = create_cls(
    "SampleData",
    obs=None,
    _obs=None,
    obs_legal=None,
    _obs_legal=None,
    act=None,
    rew=None,
    ret=None,
    done=None,
)
```

随后进行提取状态obs，_obs，动作act，合法性掩码_obs_legal，奖励rew，终止标志not_done
```python
batch_feature_vec  = [frame.obs for frame in t_data]
_batch_feature_vec = [frame._obs for frame in t_data]
batch_action       = torch.LongTensor([int(frame.act) for frame in t_data]).view(-1, 1).to(self.device)
_batch_obs_legal   = torch.stack([frame._obs_legal for frame in t_data]).bool().to(self.device)
rew      = torch.tensor([frame.rew for frame in t_data], device=self.device)
not_done = torch.tensor([1 - frame.done for frame in t_data], device=self.device)

```

- 计算当前状态下的Q值
```python
logits = self.model(batch_feature_vec)
```

- 计算目标Q值
```python
with torch.no_grad():
    q_next = self.model(_batch_feature_vec)
    q_next = q_next.masked_fill(~_batch_obs_legal, float('-inf'))  # 掩码非法动作
    q_max  = q_next.max(dim=1).values  # 最大 Q 值（下一状态的动作）

target_q = rew + self._gamma * q_max * not_done
```
计算出下一状态的所有Q值，并且将非法动作掩码，随后获取`q_max`得到下一状态的最大Q值，从而计算出DQN需要的`target_q`

- 计算损失
```python
self.optim.zero_grad()
model = getattr(self, "model")
model.train()  # 训练模式
logits = model(batch_feature) 
loss = torch.square(target_q - logits.gather(1, batch_action).view(-1)).mean() # MSE
```

- 反向传播
```python
loss.backward()
model_grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
self.optim.step()
```
通过反向传播计算梯度，更新参数，随后将信息进行上报

3. **预测函数`predict_detail`**

该函数根据当前状态数据选择一个动作，支持（epsilon-greedy） 策略，在一定的概率下随机选择动作，否则根据$Q$网络输出的$Q$值选择最佳动作。它还会处理合法性掩码，确保选择的动作是合法的。

输入为`list_obs_data`，本质为`ObsData`示例，长度为`batch`；`list_obs_data`控制是否依赖模型进行推理，如果为1，则完全依赖Q网络的最佳act

- ε-贪婪策略

根据`predict_count`和`epsilon_decay`计算当前的`epsilon`
```python
self.epsilon = self.epsilon_min + (self.epsilon_max - self.epsilon_min) * np.exp(
    -self.epsilon_decay * self.predict_count
)
```
随后使用典型的ε-贪婪策略进行选择act，再进行标准化返回`ActData`
```python
if not exploit_flag and np.random.rand(1) < self.epsilon:
    random_action = np.random.rand(batch, self.act_shape)
    random_action = torch.tensor(random_action, dtype=torch.float32).to(self.device)
    random_action = random_action.masked_fill(~legal_act, 0)
    act = random_action.argmax(dim=1).cpu().view(-1, 1).tolist()
else:
    feature = self.__convert_to_tensor(feature_vec)
    logits = model(feature)
    logits = logits.masked_fill(~legal_act, float(torch.min(logits)))
    act = logits.argmax(dim=1).cpu().view(-1, 1).tolist()
```




### **网络结构**

使用MLP计算每个动作的Q值，包含3个全连接层，具体结构：
```python
self.q_mlp = MLP([self.feature_len, 256, 128, action_shape], "q_mlp")
```
特征层22->256->128->action_space，两个隐藏层

1. 函数`make_fc_layer`使用正交初始化，有助于防止梯度在训练过程中迅速衰减或爆炸
```python
def make_fc_layer(in_features: int, out_features: int):
    # Wrapper function to create and initialize a linear layer
    # 创建并初始化一个线性层
    fc_layer = nn.Linear(in_features, out_features)

    # initialize weight and bias
    # 初始化权重及偏移量
    nn.init.orthogonal(fc_layer.weight)
    nn.init.zeros_(fc_layer.bias)

    return fc_layer
```

2. `MLP`类，接收一个列表`fc_feat_dim_list`，这个列表定义了每一层的输入和输出维度。MLP会依次添加多个全连接层，并且在每一层之后使用非线性激活函数（默认为 ReLU），前向传播函数`forward`将输入的data通过所有全连接层和激活函数，最终返回。




### **工作流程**

位于文件`train_workflow.py`

1. `workflow`函数

传入环境env，智能体agents，logger日志，monitor监控。随后设置保存时间，读取地图信息，进入无限循环：一直进行训练，保存，上传

2. `run_episodes`函数

传入参数`n_episode`，训练的总局数

```python
training_metrics = get_training_metrics()
if training_metrics:
    logger.info(f"training_metrics is {training_metrics}")
```

调用获取当前的训练指标，完整训练准备，随后重置环境，加载最新模型。调用特征处理函数得到`ObsData`，设置`done`与`max_step_no`
```python
obs_data, _ = agent.observation_process(obs, extra_info)
```

开始一个游戏回合循环，通过状态预测动作
```python
act_data, model_version = agent.predict(list_obs_data=[obs_data])
act = agent.action_process(act_data[0])
```

调用`env.step()`将预测的动作与环境交互，得到下一个状态`_obs`，与其他信息，并进行特征处理
```python
step_no, _obs, terminated, truncated, _extra_info = env.step(act)
_obs_data, reward_list = agent.observation_process(_obs, _extra_info)
reward = sum(reward_list)
```

如果回合结束，更新信息，给出奖励，构造任务帧
```python
frame = Frame(
    obs=obs_data.feature,
    _obs=_obs_data.feature,
    obs_legal=obs_data.legal_act,
    _obs_legal=_obs_data.legal_act,
    act=act,
    rew=reward,
    done=done,
    ret=reward,
)
```

